In [0]:
%fs

ls FileStore/tables

path,name,size,modificationTime
dbfs:/FileStore/tables/departments.csv,departments.csv,189,1672532940000
dbfs:/FileStore/tables/jobs__1___1_.csv,jobs__1___1_.csv,4419,1672532940000


In [0]:
dbutils.fs.mkdirs("/FileStore/Landing")



Out[1]: True

In [0]:
%fs
ls FileStore/tables

path,name,size,modificationTime
dbfs:/FileStore/tables/departments.csv,departments.csv,189,1672532940000
dbfs:/FileStore/tables/hired_employees.csv,hired_employees.csv,92933,1672586643000
dbfs:/FileStore/tables/jobs__1___1_.csv,jobs__1___1_.csv,4419,1672532940000


In [0]:
%fs 
cp FileStore/tables/hired_employees.csv /FileStore/Landing/


res5: Boolean = true

In [0]:
%fs 
cp FileStore/tables/departments.csv /FileStore/Landing/

res1: Boolean = true

In [0]:
%fs 
cp FileStore/tables/jobs__1___1_.csv /FileStore/Landing/

res2: Boolean = true

In [0]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType 
from pyspark.sql.types import ArrayType, DoubleType, BooleanType
from pyspark.sql.functions import col,array_contains

In [0]:

#Hired Employees


schema_hired_employees  = StructType() \
      .add("Id",IntegerType(),True) \
      .add("Name",StringType(),True) \
      .add("Datatime",StringType(),True) \
      .add("Department_Id",IntegerType(),True) \
      .add("Job_Id",IntegerType(),True) \
 
df_hired_employees = spark.read \
          .option("delimiter",",")\
          .option("header","false")\
          .schema(schema_hired_employees) \
          .csv("/FileStore/Landing/hired_employees.csv")
df_hired_employees.createOrReplaceTempView("tb_hired_employees")

#Departments

schema_deparments = StructType() \
      .add("id",IntegerType(),True) \
      .add("deparment",StringType(),True) \
 
df_departments = spark.read \
          .option("delimiter",",")\
          .option("header","false")\
          .schema(schema_deparments) \
          .csv("/FileStore/Landing/departments.csv")
df_departments.createOrReplaceTempView("tb_department")

#Jobs
schema_Jobs  = StructType() \
      .add("Id",IntegerType(),True) \
      .add("Job",StringType(),True) \

df_Jobs = spark.read \
          .option("delimiter",",")\
          .option("header","false")\
          .schema(schema_Jobs) \
          .csv("/FileStore/Landing/jobs__1___1_.csv")
df_Jobs.createOrReplaceTempView("tb_Jobs")


#df.display()


In [0]:
%sql

 select * from tb_department;
--  select * from tb_hired_employees;
--  select * from tb_Jobs;

id,deparment
1,Product Management
2,Sales
3,Research and Development
4,Business Development
5,Engineering
6,Human Resources
7,Services
8,Support
9,Marketing
10,Training


In [0]:
%sql
desc TB_DEPARTMENT

col_name,data_type,comment
id,int,null
deparment,string,null


In [0]:
%sql
desc TB_HIRED_EMPLOYEES 

col_name,data_type,comment
Id,int,null
Name,string,null
Datatime,string,null
Department_Id,int,null
Job_Id,int,null


In [0]:
%sql
CREATE OR REPLACE TEMP VIEW TESTHIRED_EMPLOYEE AS
SELECT 
        HE.Name,
        HE.datatime,
        MONTH(HE.datatime) as MONTH,
        CASE WHEN MONTH(HE.datatime) IN (1,2,3)    THEN 1 ELSE 0 END AS Q1,
        CASE WHEN MONTH(HE.datatime) IN (4,5,6)    THEN 1 ELSE 0 END AS Q2,
        CASE WHEN MONTH(HE.datatime) IN (7,8,9)    THEN 1 ELSE 0 END AS Q3,     
        CASE WHEN MONTH(HE.datatime) IN (10,11,12) THEN 1 ELSE 0 END AS Q4,
        --CASE WHEN MONTH(HE.datatime) IS NULL THEN 'N/I' 1 ELSE 0 END AS N/I,
        DPT.deparment AS DEPARMENT,
        JB.JOB AS JOB
 FROM   TB_HIRED_EMPLOYEES HE
 LEFT JOIN TB_DEPARTMENT DPT ON HE.DEPARTMENT_ID = DPT.id
 LEFT JOIN TB_JOBS JB ON HE.JOB_ID = JB.ID

In [0]:
%sql
SELECT
       DEPARMENT,
       JOB,
       SUM(Q1) AS Q1,
       SUM(Q2) AS Q2,
       SUM(Q3) AS Q3,
       SUM(Q4) AS Q4
FROM   TESTHIRED_EMPLOYEE
GROUP BY DEPARMENT,JOB,Q1,Q2,Q3,Q4

DEPARMENT,JOB,Q1,Q2,Q3,Q4
Training,Internal Auditor,0,0,1,0
Research and Development,Nuclear Power Engineer,0,0,2,0
Training,Geologist II,0,0,1,0
Support,Senior Financial Analyst,0,2,0,0
Training,Programmer IV,0,0,1,0
Research and Development,Structural Engineer,0,1,0,0
Sales,Software Test Engineer IV,0,1,0,0
Services,Structural Analysis Engineer,0,1,0,0
Human Resources,Executive Secretary,0,1,0,0
Engineering,Software Test Engineer IV,0,1,0,0


In [0]:
%sql
SELECT
       JOB,
       SUM(Q1) AS Q1,
       SUM(Q2) AS Q2,
       SUM(Q3) AS Q3,
       SUM(Q4) AS Q4
FROM   TESTHIRED_EMPLOYEE
GROUP BY JOB,Q1,Q2,Q3,Q4

JOB,Q1,Q2,Q3,Q4
Data Coordiator,0,0,0,3
Librarian,0,0,0,2
Senior Financial Analyst,0,0,0,5
Engineer IV,0,6,0,0
Web Developer III,0,0,1,0
Computer Systems Analyst IV,1,0,0,0
Research Associate,0,0,2,0
null,0,0,0,2
Senior Developer,6,0,0,0
Speech Pathologist,0,1,0,0


In [0]:
%sql
select * from TESTHIRED_EMPLOYEE 

Name,datatime,MONTH,Q1,Q2,Q3,Q4,DEPARMENT,JOB
Harold Vogt,2021-11-07T02:48:42Z,11,0,0,0,1,Sales,Health Coach III
Ty Hofer,2021-05-30T05:43:46Z,5,0,1,0,0,Support,null
Lyman Hadye,2021-09-01T23:27:38Z,9,0,0,1,0,Engineering,Structural Analysis Engineer
Lotti Crowthe,2021-10-01T13:04:21Z,10,0,0,0,1,Accounting,Statistician II
Gretna Lording,2021-10-10T22:22:17Z,10,0,0,0,1,Human Resources,Quality Engineer
Marlow Antecki,2021-04-23T23:45:42Z,4,0,1,0,0,Human Resources,Web Designer II
Joan Rillett,2021-10-10T01:33:31Z,10,0,0,0,1,Marketing,Recruiting Manager
Ulrick Nucciotti,2021-07-24T01:28:40Z,7,0,0,1,0,Support,Accounting Assistant III
Lucretia Northcote,2021-04-01T21:22:47Z,4,0,1,0,0,Marketing,Office Assistant III
Arty Giacobo,2022-02-08T12:27:07Z,2,1,0,0,0,Human Resources,Associate Professor


In [0]:
%sql
desc TB_HIRED_EMPLOYEES

col_name,data_type,comment
Id,int,null
Name,string,null
Datatime,string,null
Department_Id,int,null
Job_Id,int,null


In [0]:
%sql
--CREATE OR REPLACE TEMP VIEW HIRED_EMPLOYEE_ALL AS
SELECT 
        HE.Name,
        HE.datatime,
        MONTH(HE.datatime) as MONTH,
        YEAR(to_date(HE.datatime)) as YEAR,
        CASE WHEN MONTH(HE.datatime) IN (1,2,3)    THEN 1 ELSE 0 END AS Q1,
        CASE WHEN MONTH(HE.datatime) IN (4,5,6)    THEN 1 ELSE 0 END AS Q2,
        CASE WHEN MONTH(HE.datatime) IN (7,8,9)    THEN 1 ELSE 0 END AS Q3,     
        CASE WHEN MONTH(HE.datatime) IN (10,11,12) THEN 1 ELSE 0 END AS Q4,
        --CASE WHEN MONTH(HE.datatime) IS NULL THEN 'N/I' 1 ELSE 0 END AS N/I,
        DPT.deparment AS DEPARMENT,
        JB.JOB AS JOB
 FROM   TB_HIRED_EMPLOYEES HE
 LEFT JOIN TB_DEPARTMENT DPT ON HE.DEPARTMENT_ID = DPT.id
 LEFT JOIN TB_JOBS JB ON HE.JOB_ID = JB.ID

Name,datatime,MONTH,YEAR,Q1,Q2,Q3,Q4,DEPARMENT,JOB
Harold Vogt,2021-11-07T02:48:42Z,11,2021,0,0,0,1,Sales,Health Coach III
Ty Hofer,2021-05-30T05:43:46Z,5,2021,0,1,0,0,Support,null
Lyman Hadye,2021-09-01T23:27:38Z,9,2021,0,0,1,0,Engineering,Structural Analysis Engineer
Lotti Crowthe,2021-10-01T13:04:21Z,10,2021,0,0,0,1,Accounting,Statistician II
Gretna Lording,2021-10-10T22:22:17Z,10,2021,0,0,0,1,Human Resources,Quality Engineer
Marlow Antecki,2021-04-23T23:45:42Z,4,2021,0,1,0,0,Human Resources,Web Designer II
Joan Rillett,2021-10-10T01:33:31Z,10,2021,0,0,0,1,Marketing,Recruiting Manager
Ulrick Nucciotti,2021-07-24T01:28:40Z,7,2021,0,0,1,0,Support,Accounting Assistant III
Lucretia Northcote,2021-04-01T21:22:47Z,4,2021,0,1,0,0,Marketing,Office Assistant III
Arty Giacobo,2022-02-08T12:27:07Z,2,2022,1,0,0,0,Human Resources,Associate Professor


In [0]:
%sql
SELECT 
        DEPARMENT,
        JOB,
        COUNT(*) AS HIRED
 FROM   HIRED_EMPLOYEE_ALL
 WHERE  YEAR = 2021
 GROUP BY DEPARMENT,JOB
 ORDER BY HIRED DESC


DEPARMENT,JOB,HIRED
Support,Systems Administrator IV,8
Services,Programmer IV,8
Support,Recruiter,8
Engineering,Quality Engineer,6
Research and Development,Recruiter,6
Services,Teacher,6
Support,Web Designer I,6
Engineering,Web Designer I,6
Support,Clinical Specialist,6
Marketing,Statistician II,6
